In [5]:
from graph_maker import GraphMaker, Ontology, GroqClient, OpenAIClient
from graph_maker import Document


ontology = Ontology(
    # labels of the entities to be extracted. Can be a string or an object, like the following.
    labels=[
        {"Person": "Person name without any adjectives, Remember a person may be references by their name or using a pronoun"},
        {"Object": "Do not add the definite article 'the' in the object name"},
        {"Event": "Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
        "Place",
        "Document",
        "Organisation",
        "Action",
        {"Miscellanous": "Any important concept can not be categorised with any other given label"},
    ],
    # Relationships that are important for your application.
    # These are more like instructions for the LLM to nudge it to focus on specific relationships.
    # There is no guarentee that only these relationships will be extracted, but some models do a good job overall at sticking to these relations.
    relationships=[
        "Relation between any pair of Entities",
        ],
)

## Select a groq supported model
## model = "mixtral-8x7b-32768"
#model ="llama3-8b-8192"
model = "gpt-4-turbo"
## model = "llama3-70b-8192"
## model="gemma-7b-it" ## This is probably the fastest of all models, though a tad inaccurate.

llm = OpenAIClient(model=model, temperature=0.1, top_p=0.5)
graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

docs = [Document(text="James works at openai in it's alignment team that also contains james' friend mark", metadata={"key": "1"})]
print(docs)
print(list(docs))
## create a graph out of a list of Documents.
graph = graph_maker.from_documents(
    list(docs),
    delay_s_between=10 ## delay_s_between because otherwise groq api maxes out pretty fast.
    )
## result -> a list of Edges.
print("Total number of Edges", len(graph))
## 1503


▶︎ GRAPH MAKER LOG - 2024-05-14 21:51:32 - INFO 
Document: 1


▶︎ GRAPH MAKER VERBOSE - 2024-05-14 21:51:32 - INFO 
Using Ontology:
labels=[{'Person': 'Person name without any adjectives, Remember a person may be references by their name or using a pronoun'}, {'Object': "Do not add the definite article 'the' in the object name"}, {'Event': 'Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc.'}, 'Place', 'Document', 'Organisation', 'Action', {'Miscellanous': 'Any important concept can not be categorised with any other given label'}] relationships=['Relation between any pair of Entities']



[Document(text="James works at openai in it's alignment team that also contains james' friend mark", metadata={'key': '1'})]
[Document(text="James works at openai in it's alignment team that also contains james' friend mark", metadata={'key': '1'})]
Using Model:  gpt-4-turbo


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-05-14 21:51:44 - INFO 
LLM Response:
[
    {
        "node_1": {"label": "Person", "name": "James"},
        "node_2": {"label": "Organisation", "name": "openai"},
        "relationship": "James is employed by openai"
    },
    {
        "node_1": {"label": "Organisation", "name": "openai"},
        "node_2": {"label": "Miscellanous", "name": "alignment team"},
        "relationship": "alignment team is a part of openai"
    },
    {
        "node_1": {"label": "Person", "name": "James"},
        "node_2": {"label": "Miscellanous", "name": "alignment team"},
        "relationship": "James is a member of the alignment team"
    },
    {
        "node_1": {"label": "Person", "name": "Mark"},
        "node_2": {"label": "Miscellanous", "name": "alignment team"},
        "relationship": "Mark is a member of the alignment team"
    },
    {
        "node_1": {"label": "

Total number of Edges 5


In [6]:
print(graph)

[Edge(node_1=Node(label='Person', name='James'), node_2=Node(label='Organisation', name='openai'), relationship='James is employed by openai', metadata={'key': '1'}, order=0), Edge(node_1=Node(label='Organisation', name='openai'), node_2=Node(label='Miscellanous', name='alignment team'), relationship='alignment team is a part of openai', metadata={'key': '1'}, order=0), Edge(node_1=Node(label='Person', name='James'), node_2=Node(label='Miscellanous', name='alignment team'), relationship='James is a member of the alignment team', metadata={'key': '1'}, order=0), Edge(node_1=Node(label='Person', name='Mark'), node_2=Node(label='Miscellanous', name='alignment team'), relationship='Mark is a member of the alignment team', metadata={'key': '1'}, order=0), Edge(node_1=Node(label='Person', name='James'), node_2=Node(label='Person', name='Mark'), relationship='Mark is a friend of James', metadata={'key': '1'}, order=0)]


In [13]:
from graph_maker import Neo4jGraphModel
create_indices = False
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)
neo4j_graph.save()

aenter called


ServiceUnavailable: Couldn't connect to localhost:8000 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 8000)) (reason [Errno 111] Connection refused)